In [8]:
import os

import pandas as pd
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

from flask import Flask, jsonify, render_template
app = Flask(__name__)


In [9]:

#################################################
# Database Setup
#################################################
dbfile = os.path.join('gun_violence.sqlite')
engine = create_engine(f"sqlite:///{dbfile}") 

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)


In [10]:

# Save references to tables
Gun_sales = Base.classes.gun_sales
Gun_law_grade= Base.classes.gun_law_grade
Mass_shootings = Base.classes.mass_shootings
Nationwide_murder_rates = Base.classes.Nationwide_murder_rates
State_laws_by_year = Base.classes.State_laws_by_year

# Create our session (link) from Python to the DB
session = Session(engine)
state_dropdown= "California"

In [11]:
gun_sales_stmt = session.query(Gun_sales).statement
gun_sales_df = pd.read_sql_query(gun_sales_stmt, session.bind)
gun_sales_by_state= gun_sales_df.loc[gun_sales_df['state']== state_dropdown,:]


#dataframe for graph
gun_sales_by_year= gun_sales_by_state.groupby(['year'])

#dataframe for chloropleth
gun_sales_2016= gun_sales_df.loc[gun_sales_df['year']== 2016,:]
gun_sales_2016.set_index('state', inplace=True)

#stats for state facts
gun_sales_state= gun_sales_2016.loc[state_dropdown,'totals']
print(gun_sales_state)

1326612


In [51]:
mass_shootings_stmt = session.query(Mass_shootings).statement
mass_shootings_df = pd.read_sql_query(mass_shootings_stmt, session.bind)
mass_shootings_by_state= mass_shootings_df.loc[mass_shootings_df['state']== state_dropdown,:]
mass_shootings_by_year= mass_shootings_by_state[['id','state','year','fatalities','injured', 'total_victims']]


In [55]:
#data for chloropleth
chloro_df = mass_shootings_df.query('year == 2016').groupby(['state'])[['fatalities']].sum().reset_index()
chloro_df.head()

,state,fatalities
0,Alabama,5
1,Arizona,8
2,California,6
3,Delaware,1
4,Florida,61


In [60]:
#stats for state facts
chloro_df.reset_index()
mass_shooting_deaths= chloro_df.loc[state_dropdown, 'fatalities']
print(mass_shooting_deaths)


6


In [61]:
mass_sum = mass_shootings_df.groupby(['year','state'])[['fatalities']].sum().reset_index()
mass_count = mass_shootings_df.groupby(['year','state'])[['fatalities']].count().reset_index()

In [62]:
#dataframe for graph
mass_df = pd.merge(mass_sum, mass_count, on = 'year',how ='inner')
mass2_df = mass_df.drop('state_y', axis=1)
mass2_df.head()

,year,state_x,fatalities_x,fatalities_y
0,1966,Arizona,5,1
1,1966,Arizona,5,1
2,1966,Texas,17,1
3,1966,Texas,17,1
4,1971,Washington,2,1


In [22]:
murder_rate_stmt = session.query(Nationwide_murder_rates).statement
murder_rate_df = pd.read_sql_query(murder_rate_stmt, session.bind)
murder_rate_df.head()


#dataframe for chloropleth
murder_rate_2016= murder_rate_df[['State','Y2016']]

#dataframe for graph
murder_rate_by_state= murder_rate_df.loc[murder_rate_df['State']== state_dropdown,:]
print(murder_rate_by_state)

#stats for state facts
murder_rate_df.set_index('State', inplace=True)
murder_rate_state= murder_rate_df.loc[state_dropdown,'Y2016']



    id       State  Y2016  Y2015  Y2014  Y2013  Y2012  Y2011  Y2010  Y2009  \
25  26  California    4.9    4.8    4.4    4.6    5.0    4.8    4.8    5.3   

    ...    Y2005  Y2004  Y2003  Y2002  Y2001  Y2000  Y1999  Y1998  Y1997  \
25  ...      6.9    6.7    6.8    6.8    6.4    6.1    6.0    6.6    8.0   

    Y1996  
25    9.1  

[1 rows x 23 columns]


In [76]:
gun_grade_stmt = session.query(Gun_law_grade).statement
gun_grade_df = pd.read_sql_query(gun_grade_stmt, session.bind)

#data for choloropleth
gun_grade_chloro= gun_grade_df[['state','grade']]
gun_grade_chloro.head()

,state,grade
0,Mississippi,F
1,Kansas,F
2,Missouri,F
3,Arizona,F
4,Idaho,F


In [77]:
#data for choloropleth
gun_grade_chloro= gun_grade_df[['state','grade']]
gun_grade_chloro.head()



,state,grade
0,Mississippi,F
1,Kansas,F
2,Missouri,F
3,Arizona,F
4,Idaho,F


In [80]:
#stats for state facts
gun_grade_df.set_index('state', inplace=True)
state_grade= gun_grade_df.loc[state_dropdown, 'grade']
print(state_grade)

A
